<a href="https://colab.research.google.com/github/Maryam512-math/My-Assignment-/blob/main/Copy_of_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Set paths
train_dir = "/content/drive/MyDrive/MY NEW TESTDATASET"
# Update this path to your actual validation directory
val_dir = "/content/drive/MyDrive/MY NEW TESTDATASET" # Replace with your validation directory path

# Data generators
train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir, target_size=(128, 128), batch_size=32, class_mode='categorical') # Changed to categorical

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir, target_size=(128, 128), batch_size=32, class_mode='categorical') # Changed to categorical

# Build CNN model
# Number of classes should be equal to the number of classes found by flow_from_directory
num_classes = train_gen.num_classes # Get number of classes from the generator

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')  # Changed to softmax for multiclass
])

# Compile & train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # Changed loss to categorical_crossentropy
model.fit(train_gen, epochs=10, validation_data=val_gen)

# Save model
model.save("image_cnn_model.h5")

Found 2 images belonging to 3 classes.
Found 2 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5000 - loss: 1.0581

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 1.0581 - val_accuracy: 0.5000 - val_loss: 0.4855
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.5000 - loss: 0.6116 - val_accuracy: 1.0000 - val_loss: 0.1459
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 1.0000 - loss: 0.1639 - val_accuracy: 1.0000 - val_loss: 0.0202
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step - accuracy: 1.0000 - loss: 0.0217 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 1.4590e-04
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 1.0000 - loss: 1.5973e-04 - val_accuracy: 1.0000 - val_loss: 1.4186e-05
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 1.0000 - loss: 2.7179e-05 - val_accuracy: 1.0000 - val_loss: 1.7285e-06
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - accuracy: 1.0000 - loss: 7.7486e-07 - val_accuracy: 1.0000 -

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the model
model = tf.keras.models.load_model("image_cnn_model.h5")

# Streamlit UI
st.title("Image Classification with CNN")
st.write("Upload an image and the model will predict its class.")

# Upload image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

# Process and predict
if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Preprocess
    img = image.resize((128, 128))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict
    prediction = model.predict(img_array)[0][0]
    pred_class = "Class 1" if prediction >= 0.5 else "Class 0"
    st.success(f"Prediction: {pred_class} (Confidence: {prediction:.2f})")


Overwriting app.py


In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image

# Load pre-trained CNN model
@st.cache_resource
def load_model():
    return tf.keras.models.load_model("image_cnn_model.h5")

model = load_model()

# Set image size (must match what the model was trained on)
IMG_SIZE = (128, 128)

# Streamlit UI
st.title("🧠 CNN Image Classifier")
st.write("Upload an image and the CNN model will classify it.")

# Upload image
uploaded_file = st.file_uploader("Upload an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display image
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Preprocess image
    img_resized = image.resize(IMG_SIZE)
    img_array = np.array(img_resized) / 255.0  # normalize
    img_array = np.expand_dims(img_array, axis=0)  # shape: (1, 128, 128, 3)

    # Predict
    prediction = model.predict(img_array)[0][0]

    # Class decision (assuming binary classification)
    predicted_class = "Class 1" if prediction >= 0.5 else "Class 0"
    confidence = prediction if prediction >= 0.5 else 1 - prediction

    st.markdown(f"### 🔍 Prediction: **{predicted_class}**")
    st.markdown(f"### 🧪 Confidence: **{confidence:.2f}**")


2025-07-15 12:23:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 12:23:09.040 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 12:23:09.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 12:23:09.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 12:23:09.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 12:23:09.046 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 12:23:09.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 12:23:09.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
streamlit run app.py

SyntaxError: invalid syntax (ipython-input-17-3737097518.py, line 1)

In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.2 MB/s eta 0:00:00


In [ ]:
# Define the dependencies
dependencies = [
    "streamlit",
    "scikit-learn",
    "pandas",
    "numpy"
]

# Create and write to requirements.txt
with open("requirements.txt", "w") as file:
    for package in dependencies:
        file.write(package + "\n")

print("requirements.txt file created successfully.")


requirements.txt file created successfully.
